In [17]:
import importlib
import sys
from pathlib import Path
import os
import json
import pandas as pd
import numpy as np

In [5]:
project_root = Path.cwd().parents[1]
src_path = project_root / "src"
sys.path.insert(0, str(project_root))

In [3]:
from src.config import llm_config
importlib.reload(llm_config)
from src.config.llm_config import get_llm

## Test data loader

In [ ]:
from src.agents.nodes import data_loader_agent
importlib.reload(data_loader_agent)
from src.agents.nodes.data_loader_agent import DataLoader

In [20]:
print("\n" + "="*60)
print("Testing Agent 1: Data Loader")
print("="*60)


Testing Agent 1: Data Loader


In [21]:
llm = get_llm()
agent = DataLoader(llm)

In [22]:
csv_path = os.path.join(project_root, 'inputs', 'dummy_options.csv')
csv_path = os.path.abspath(csv_path)
csv_path

'/Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv'

In [23]:
print(f"\nTest CSV path: {csv_path}")
print(f"File exists: {os.path.exists(csv_path)}")


Test CSV path: /Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv
File exists: True


In [ ]:
initial_state = {
    "messages": [],
    "csv_file_path": csv_path,
    "csv_data": None,
    "data_loader_agent_status": None,
    "calculation_results": None,
    "greeks_data": None,
    "calculator_agent_status": None,
    "test_results": None,
    "test_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": None,
    "workflow_status": "started",
    "errors": []
}

# Run agent
print("\n" + "-"*60)
print("Running Data Loader Agent...")
print("-"*60)


------------------------------------------------------------
Running Data Loader Agent...
------------------------------------------------------------


In [25]:
result = agent(initial_state)

In [27]:
print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('agent1_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")


Agent execution completed!

Agent Status: completed
Current Agent: agent1
Workflow Status: in_progress


## Test calculator

In [9]:
from src.agents.nodes import data_loader_agent
importlib.reload(data_loader_agent)
from src.agents.nodes.calculator_agent import Calculator

In [10]:
print("\n" + "="*60)
print("Testing Agent 2: Calculator")
print("="*60)


Testing Agent 2: Calculator


In [11]:
llm = get_llm()
agent = Calculator(llm)

In [ ]:
mock_csv_data = {
        "option_type": {0: "call", 1: "put"},
        "S": {0: 100.0, 1: 105.0},
        "K": {0: 100.0, 1: 100.0},
        "T": {0: 1.0, 1: 0.5},
        "r": {0: 0.05, 1: 0.05},
        "sigma": {0: 0.2, 1: 0.25}
    }
initial_state = {
    "messages": [],
    "csv_file_path": "/inputs/dummy_options.csv",
    "csv_data": mock_csv_data,
    "data_loader_agent_status": "completed",
    "calculation_results": None,
    "greeks_data": None,
    "calculator_agent_status": None,
    "test_results": None,
    "test_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": None,
    "workflow_status": "started",
    "errors": []
}

# Run agent
print("\n" + "-"*60)
print("Running Calculator Agent...")
print("-"*60)
print(f"\nInput CSV Data:")
print(json.dumps(mock_csv_data, indent=2))


------------------------------------------------------------
Running Calculator Agent...
------------------------------------------------------------

Input CSV Data:
{
  "option_type": {
    "0": "call",
    "1": "put"
  },
  "S": {
    "0": 100.0,
    "1": 105.0
  },
  "K": {
    "0": 100.0,
    "1": 100.0
  },
  "T": {
    "0": 1.0,
    "1": 0.5
  },
  "r": {
    "0": 0.05,
    "1": 0.05
  },
  "sigma": {
    "0": 0.2,
    "1": 0.25
  }
}


In [13]:
result = agent(initial_state)

print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('calculator_agent_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")

Testing: Running sensitivity test for call with base S=100.0, K=100.0, T=1.0, r=0.05, sigma=0.2

Agent execution completed!

Agent Status: completed
Current Agent: calculator
Workflow Status: in_progress


## Test testor

### test_batch_greeks_validator

In [26]:
from src.core.bsm_validator import batch_greeks_validator

In [6]:
print("\n" + "="*70)
print("TEST 1: Batch Greeks Validator")
print("="*70)


TEST 1: Batch Greeks Validator


In [7]:
sample_data = {
    "option_type": {0: "call", 1: "put", 2: "call"},
    "S": {0: 100.0, 1: 100.0, 2: 105.0},
    "K": {0: 100.0, 1: 100.0, 2: 100.0},
    "T": {0: 1.0, 1: 1.0, 2: 0.5},
    "r": {0: 0.05, 1: 0.05, 2: 0.05},
    "sigma": {0: 0.2, 1: 0.2, 2: 0.25}
}

#### Step 1: _parse_csv_data

In [8]:
csv_data = sample_data

In [10]:
[isinstance(csv_data, str),isinstance(csv_data, dict)]

[False, True]

In [18]:
data = csv_data
df = pd.DataFrame(data)

#### Step 2

In [20]:
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing = [c for c in required_cols if c not in df.columns]
missing

[]

In [21]:
results = {
    "total_options": len(df),
    "passed": 0,
    "failed": 0,
    "errors": 0,
    "details": []
}

In [22]:
idx, row = list(df.iterrows())[0]

In [24]:
option_type = str(row['option_type']).lower()
S, K, T, r, sigma = float(row['S']), float(row['K']), float(row['T']), float(row['r']), float(row['sigma'])

In [27]:
result_json = batch_greeks_validator.invoke({"csv_data": sample_data})
result = json.loads(result_json)

In [28]:
print("\n" + "-"*70)
print("Validation Results:")
print("-"*70)
print(f"Status: {result['status']}")
print(f"Total Options: {result['total_options']}")
print(f"Passed: {result['passed']}")
print(f"Failed: {result['failed']}")
print(f"Errors: {result['errors']}")


----------------------------------------------------------------------
Validation Results:
----------------------------------------------------------------------
Status: passed
Total Options: 3
Passed: 3
Failed: 0
Errors: 0


### test_validate_put_call_parity

In [31]:
from src.core.bsm_validator import validate_put_call_parity

In [32]:
print("\n" + "="*70)
print("TEST 2: Put-Call Parity Validator")
print("="*70)


TEST 2: Put-Call Parity Validator


In [33]:
sample_data = {
    "option_type": {0: "call", 1: "put", 2: "call", 3: "put"},
    "S": {0: 100.0, 1: 100.0, 2: 105.0, 3: 105.0},
    "K": {0: 100.0, 1: 100.0, 2: 100.0, 3: 100.0},
    "T": {0: 1.0, 1: 1.0, 2: 0.5, 3: 0.5},
    "r": {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.05},
    "sigma": {0: 0.2, 1: 0.2, 2: 0.25, 3: 0.25}
}

In [34]:
result_json = validate_put_call_parity.invoke({"csv_data": sample_data})
result = json.loads(result_json)

print("\n" + "-"*70)
print("Parity Test Results:")
print("-"*70)
print(f"Status: {result['status']}")
print(f"Pairs Found: {result['pairs_found']}")
print(f"Pairs Tested: {result['pairs_tested']}")
print(f"Pairs Passed: {result['pairs_passed']}")
print(f"Pairs Failed: {result['pairs_failed']}")


----------------------------------------------------------------------
Parity Test Results:
----------------------------------------------------------------------
Status: passed
Pairs Found: 2
Pairs Tested: 2
Pairs Passed: 2
Pairs Failed: 0


### test_validate_sensitivity

In [35]:
from src.core.bsm_validator import validate_sensitivity

In [36]:
print("\n" + "="*70)
print("TEST 3: Sensitivity Validator")
print("="*70)


TEST 3: Sensitivity Validator


In [37]:
sample_data = {
    "option_type": {0: "call"},
    "S": {0: 100.0},
    "K": {0: 100.0},
    "T": {0: 1.0},
    "r": {0: 0.05},
    "sigma": {0: 0.2}
}

In [38]:
result_json = validate_sensitivity.invoke({"csv_data": sample_data})
result = json.loads(result_json)

print("\n" + "-"*70)
print("Sensitivity Test Results:")
print("-"*70)
print(f"Status: {result['status']}")


----------------------------------------------------------------------
Sensitivity Test Results:
----------------------------------------------------------------------
Status: passed


### Test testor

In [37]:
from src.agents.nodes import tester_agent
importlib.reload(tester_agent)
from src.agents.nodes.tester_agent import Tester

In [38]:
print("\n" + "="*60)
print("Testing Agent 3: Tester (Test Agent)")
print("="*60)


Testing Agent 3: Tester (Test Agent)


In [41]:
llm = get_llm()
agent = Tester(llm)

In [ ]:
mock_csv_data = {
    "option_type": {0: "call", 1: "put", 2: "call", 3: "put"},
    "S": {0: 100.0, 1: 100.0, 2: 105.0, 3: 105.0},
    "K": {0: 100.0, 1: 100.0, 2: 100.0, 3: 100.0},
    "T": {0: 1.0, 1: 1.0, 2: 0.5, 3: 0.5},
    "r": {0: 0.05, 1: 0.05, 2: 0.05, 3: 0.05},
    "sigma": {0: 0.2, 1: 0.2, 2: 0.25, 3: 0.25}
}

# mock_calculation_results = """
# |   K |   S |   T | option_type   |    r |   sigma |   BSM_Price |
# |----:|----:|----:|:--------------|-----:|--------:|------------:|
# | 100 | 100 | 1   | call          | 0.05 |    0.2  |    10.4506  |
# | 100 | 100 | 1   | put           | 0.05 |    0.2  |     5.57353 |
# | 100 | 105 | 0.5 | call          | 0.05 |    0.25 |    11.4774  |
# | 100 | 105 | 0.5 | put           | 0.05 |    0.25 |     4.00839 |
# """


In [ ]:
initial_state = {
    "messages": [],
    "csv_file_path": "/inputs/dummy_options.csv",
    "csv_data": mock_csv_data,
    "data_loader_agent_status": "completed",
    # "calculation_results": mock_calculation_results,
    "calculation_results": None,
    "greeks_data": None,
    "sensitivity_data": None,
    "calculator_agent_status": "completed",
    "test_results": None,
    "tester_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": "agent2",
    "workflow_status": "in_progress",
    "errors": []
}
print("\n" + "-"*60)
print("Running Test Agent...")
print("-"*60)
print("\nThis agent will execute actual pytest test functions:")
print("  - test_greeks_call_basic")
print("  - test_greeks_put_call_parity")
print("  - test_sensitivity_length_and_fields")


------------------------------------------------------------
Running Test Agent...
------------------------------------------------------------

This agent will execute actual pytest test functions:
  - test_greeks_call_basic
  - test_greeks_put_call_parity
  - test_sensitivity_length_and_fields


In [44]:
result = agent(initial_state)

print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('tester_agent_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")


Agent execution completed!

Agent Status: completed
Current Agent: tester
Workflow Status: in_progress


## Test summary writer